In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
import re
from datetime import date
import time 

# ignore SettingWithCopyWarning:
pd.options.mode.chained_assignment = None

In [3]:
# create folder for the HTMLs
folder_html = 'Gold_de'
if not os.path.exists(folder_html):
    os.makedirs(folder_html)

In [4]:
# create folder for the csvs
folder_csv = 'Gold_de_csv'
if not os.path.exists(folder_csv):
    os.makedirs(folder_csv)

In [5]:
date_today = date.today()

In [6]:
file_name = 'gold_de_'
end_html = '.html'
end_csv = '.csv'

In [7]:
file_name + date_today.strftime('%d.%m.%Y') + end_html

'gold_de_13.05.2020.html'

In [8]:
# get the html with request
# save the html in the folder
url = 'https://www.gold.de/aufgeldtabelle/silber/'
r = requests.get(url)
with open(os.path.join(folder_html, file_name + date_today.strftime('%d.%m.%Y') + end_html), mode = 'w') as file:
    file.write(r.text)

In [9]:
path_html = folder_html + '/' + file_name + date_today.strftime('%d.%m.%Y') + end_html

In [10]:
df_list = pd.read_html(path_html)

In [11]:
df_list[0]

,0,1,2,3,4
0,NaN,USD / oz,EUR / oz,EUR / g,EUR / kg
1,GOLD,1.70798,1.57043,5049,50.49037
2,SILBER,1554,1429,046,45939


In [12]:
df_list[1]

,Rang,Produktbezeichnung,Produktpreis,Preis/kg,Preis/oz,Preis/g,Aufgeld in %,Aufgeld in %.1,Unnamed: 8
0,"Produkte: Münzen Barren Münzbarren, TafelnBudg...","Produkte: Münzen Barren Münzbarren, TafelnBudg...","Produkte: Münzen Barren Münzbarren, TafelnBudg...","Silberspot/kg459,39 EUR","Silberspot/oz14,29 EUR","Silberspot/g0,46 EUR",NaN,NaN,NaN
1,1,Silberbarren 1000 ozGold & Silver World,"17.011,84 EUR","607,60 EUR","18,90 EUR","0,61 EUR","32,26 %",NaN,NaN
2,2,Silberbarren 1 kgcoininvest.com,"625,11 EUR","625,10 EUR","19,44 EUR","0,63 EUR","36,07 %",NaN,NaN
3,3,Münzbarren 5 kgMP Edelmetalle,"3.156,13 EUR","631,20 EUR","19,63 EUR","0,63 EUR","37,40 %",NaN,NaN
4,4,Maple Leaf 1 ozSilber-Corner.de,"19,65 EUR","631,80 EUR","19,65 EUR","0,63 EUR","37,53 %",NaN,NaN
...,...,...,...,...,...,...,...,...,...
248,246,Schwan 5 ozAuragentum.de,"349,00 EUR","2.244,20 EUR","69,80 EUR","2,24 EUR","388,52 %",NaN,NaN
249,247,"Comic, Kino & TV (Silber) 19,98 gmuenzdiscount.de","44,90 EUR","2.247,20 EUR","69,90 EUR","2,25 EUR","389,17 %",NaN,NaN
250,248,Evolution - Wonderful World 1 ozGeiger Edelmet...,"70,00 EUR","2.250,60 EUR","70,00 EUR","2,25 EUR","389,91 %",NaN,NaN
251,249,Niue Lunar Serie 5 ozMünzland,"355,00 EUR","2.282,70 EUR","71,00 EUR","2,28 EUR","396,90 %",NaN,NaN


In [13]:
df = df_list[1]

### Cleaning steps

In [14]:
df = df.iloc[1:]

In [15]:
df.dropna(axis = 1, how = 'all', inplace = True)

In [16]:
df.dropna(axis = 0, how = 'all', inplace = True)

In [17]:
# drop unrelevant columns
if 'Aufgeld in %.1' in df.columns.values:
    df.drop(columns = 'Aufgeld in %.1', inplace = True)

In [18]:
df

,Rang,Produktbezeichnung,Produktpreis,Preis/kg,Preis/oz,Preis/g,Aufgeld in %
1,1,Silberbarren 1000 ozGold & Silver World,"17.011,84 EUR","607,60 EUR","18,90 EUR","0,61 EUR","32,26 %"
2,2,Silberbarren 1 kgcoininvest.com,"625,11 EUR","625,10 EUR","19,44 EUR","0,63 EUR","36,07 %"
3,3,Münzbarren 5 kgMP Edelmetalle,"3.156,13 EUR","631,20 EUR","19,63 EUR","0,63 EUR","37,40 %"
4,4,Maple Leaf 1 ozSilber-Corner.de,"19,65 EUR","631,80 EUR","19,65 EUR","0,63 EUR","37,53 %"
5,5,Eule von Athen 1 ozMünzen Engel,"19,72 EUR","634,00 EUR","19,72 EUR","0,63 EUR","38,01 %"
...,...,...,...,...,...,...,...
248,246,Schwan 5 ozAuragentum.de,"349,00 EUR","2.244,20 EUR","69,80 EUR","2,24 EUR","388,52 %"
249,247,"Comic, Kino & TV (Silber) 19,98 gmuenzdiscount.de","44,90 EUR","2.247,20 EUR","69,90 EUR","2,25 EUR","389,17 %"
250,248,Evolution - Wonderful World 1 ozGeiger Edelmet...,"70,00 EUR","2.250,60 EUR","70,00 EUR","2,25 EUR","389,91 %"
251,249,Niue Lunar Serie 5 ozMünzland,"355,00 EUR","2.282,70 EUR","71,00 EUR","2,28 EUR","396,90 %"


In [19]:
df.drop(index = df[df['Rang']=="Gesamte Aufgeldtabelle anzeigen"].index.values, inplace = True)

In [20]:
df.rename(columns = {'Produktbezeichnung':'Product', 'Produktpreis':'Price in EUR', 'Preis/kg':'Price/kg', 
                     'Preis/oz':'Price/oz', 'Preis/g':'Price/g', 'Aufgeld in %':'Premiums in %'}, 
         inplace = True)

In [23]:
df['Price in EUR'] = df['Price in EUR'].map(lambda x: str(x).replace('EUR', '').strip().replace('.', '').replace(',','.'))
df['Price/kg'] = df['Price/kg'].map(lambda x: str(x).replace('EUR', '').strip().replace('.', '').replace(',','.'))
df['Price/oz'] = df['Price/oz'].map(lambda x: str(x).replace('EUR', '').strip().replace('.', '').replace(',','.'))
df['Price/g'] = df['Price/g'].map(lambda x: str(x).replace('EUR', '').strip().replace('.', '').replace(',','.'))
df['Premiums in %'] = df['Premiums in %'].map(lambda x: str(x).replace('%', '').strip().replace('.', '').replace(',','.'))

In [24]:
df['Price in EUR'] = df['Price in EUR'].astype('float')
df['Price/kg'] = df['Price/kg'].astype('float')
df['Price/oz'] = df['Price/oz'].astype('float')
df['Price/g'] = df['Price/g'].astype('float')
df['Premiums in %'] = df['Premiums in %'].astype('float')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 1 to 252
Data columns (total 7 columns):
Rang             250 non-null object
Product          250 non-null object
Price in EUR     250 non-null float64
Price/kg         250 non-null float64
Price/oz         250 non-null float64
Price/g          250 non-null float64
Premiums in %    250 non-null float64
dtypes: float64(5), object(2)
memory usage: 15.6+ KB


In [26]:
path_to_csv = folder_csv + '/' + file_name + date_today.strftime('%d.%m.%Y') + end_csv

In [27]:
df.to_csv(path_to_csv, index = None)

In [28]:
df

,Rang,Product,Price in EUR,Price/kg,Price/oz,Price/g,Premiums in %
1,1,Silberbarren 1000 ozGold & Silver World,17011.84,607.6,18.90,0.61,32.26
2,2,Silberbarren 1 kgcoininvest.com,625.11,625.1,19.44,0.63,36.07
3,3,Münzbarren 5 kgMP Edelmetalle,3156.13,631.2,19.63,0.63,37.40
4,4,Maple Leaf 1 ozSilber-Corner.de,19.65,631.8,19.65,0.63,37.53
5,5,Eule von Athen 1 ozMünzen Engel,19.72,634.0,19.72,0.63,38.01
...,...,...,...,...,...,...,...
248,246,Schwan 5 ozAuragentum.de,349.00,2244.2,69.80,2.24,388.52
249,247,"Comic, Kino & TV (Silber) 19,98 gmuenzdiscount.de",44.90,2247.2,69.90,2.25,389.17
250,248,Evolution - Wonderful World 1 ozGeiger Edelmet...,70.00,2250.6,70.00,2.25,389.91
251,249,Niue Lunar Serie 5 ozMünzland,355.00,2282.7,71.00,2.28,396.90


### Extract infos from the Product column 

In [29]:
# use sample function to check which expressions for weight are used
df['Product'].sample(n=20)

212                   China Panda 1/2 ozLeihhaus Lohmann
12                  Lunar Serie III 1 kgSilver-to-go.com
210                 Lunar Serie I 1/2 ozLeihhaus Lohmann
107                Niue Lunar Serie 1 ozLeihhaus Lohmann
84                       Kookaburra 2 ozLeihhaus Lohmann
15                        Krügerrand 1 ozAnlagegold24.de
39                   Lunar Serie II 1 kgSilber-Corner.de
19                                     Koala 1 kgmyVALOR
109                             Ruanda 1 ozAuragentum.de
60                    America the Beautiful 5 ozMünzland
157        Cook Islands 1/4 ozGÖBEL Münzen & Edelmetalle
203                Morgan Dollar 24,06 gmuenzdiscount.de
216                     Lunar Serie I (RAM) 5 ozMünzland
7                       Birds of Prey 1 ozMP Edelmetalle
80             Trichternetzspinne 1 ozKleiner Münzhandel
46                    Mandala Tschad 1 ozBellmann Münzen
108    Ruanda Lunar Serie 1 ozGÖBEL Münzen & Edelmetalle
228                        Star

In [30]:
df[['0', '1']] = df['Product'].str.extract(r'(\d|\d{2}|\d{2},\d{2}|\d\/\d|\d{2},\d|\d{3}|\d{4})\s(oz|kg|g|DM)')

In [31]:
df['Weight'] = df['0'] + ' ' + df['1']

In [32]:
df.drop(columns= ['0', '1'], inplace = True)

In [33]:
df['Product_name'] = df['Product'].str.extract(r'([^0-9]+)')

In [34]:
df['Product_name'] = df['Product_name'].map(lambda x: str(x).strip())

In [35]:
df.iloc[208]

Rang                                                   209
Product          Territory of Tokelau 1 ozmuenzdiscount.de
Price in EUR                                            45
Price/kg                                            1446.8
Price/oz                                                45
Price/g                                               1.45
Premiums in %                                       214.94
Weight                                                1 oz
Product_name                          Territory of Tokelau
Name: 211, dtype: object

In [36]:
products_list = df['Product'].to_list()

In [37]:
page = []
for s in products_list:
    string = str(s).split(r'\d')[-1]
    page.append(string)

In [38]:
test = 'The Queens Beasts 2 ozMünzen Engel con analagegold24.de'

In [39]:
test.split(r'(\d|\d{2}|\d{2},\d{2}|\d\/\d|\d{2},\d|\d{3})')

['The Queens Beasts 2 ozMünzen Engel con analagegold24.de']

In [40]:
re.split(r'(\d|\d{2}|\d{2},\d{2}|\d\/\d|\d{2},\d|\d{3})\s(oz|kg|g|DM)', test)

['The Queens Beasts ', '2', 'oz', 'Münzen Engel con analagegold24.de']

In [43]:
page = []
for i in range(250):
    strings = re.split(r'(\d|\d{2}|\d{2},\d{2}|\d\/\d|\d{2},\d|\d{3})\s(oz|kg|g|DM)', str(products_list[i]))
    page.append(strings)

In [44]:
page_specific = []
for i in range(250):
    strings = page[i][-1]
    page_specific.append(strings)

In [45]:
page_specific

['Gold & Silver World',
 'coininvest.com',
 'MP Edelmetalle',
 'Silber-Corner.de',
 'Münzen Engel',
 'Chiemgauer Edelmetallhandel',
 'MP Edelmetalle',
 'MP Edelmetalle',
 'MP Edelmetalle',
 'MP Edelmetalle',
 'silber-werte.de',
 'Silver-to-go.com',
 'Agosi',
 'Anlagegold24.de',
 'Anlagegold24.de',
 'ESG Edelmetalle',
 'myVALOR',
 'Auragentum.de',
 'myVALOR',
 ')Heubach Edelmetalle',
 'ESG Edelmetalle',
 'Münzen Müller',
 'silber-werte.de',
 'heraeus-gold.de',
 ')Heubach Edelmetalle',
 ')Agosi',
 ')MP Edelmetalle',
 'SGV Ohnheiser',
 'Aschhoff Edelmetalle',
 'Heubach Edelmetalle',
 'silber-werte.de',
 'Gold & Silver World',
 'Auragentum.de',
 'Auragentum.de',
 'Kronwitter GbR',
 ')Geiger Edelmetalle',
 'Silber-Corner.de',
 'ESG Edelmetalle',
 ')Heubach Edelmetalle',
 ')GoldSilberShop.de',
 'Auragentum.de',
 'myVALOR',
 'Faller Edelmetalle',
 'Bellmann Münzen',
 'silber-werte.de',
 'Bellmann Münzen',
 'Heubach Edelmetalle',
 'Kronwitter GbR',
 'Silverbroker.de',
 'silber-werte.de',
 'sil